In [6]:
# %%capture
# !pip install unsloth
# # Also get the latest nightly Unsloth!
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [7]:
# %pip uninstall unsloth unsloth_zoo -y
# %pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# %pip install --upgrade --no-cache-dir "git+https://github.com/unslothai/unsloth-zoo.git"

In [8]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [9]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.2.4: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-PCIE-40GB. Max memory: 39.394 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 8.0. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "v_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [11]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("passionMan/diabetes_v18", split = "train")
dataset = dataset.filter(lambda example: example['task'] in ['summarization', 'generation', 'daily_diets', 'alternative_diet'])
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 8,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 1,
        warmup_steps = 50,
        num_train_epochs = 3, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 100,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs_new/generation",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=8):   0%|          | 0/3050 [00:00<?, ? examples/s]

In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,050 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 1
\        /    Total batch size = 32 | Total steps = 288
 "-____-"     Number of trainable parameters = 6,815,744


Step,Training Loss
100,1.891800
200,1.032700


In [26]:
import json

import time

def load_json(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

def save_to_jsonl(file_path, data):
    with open(file_path, "a", encoding="utf-8") as f: 
        f.write(json.dumps(data, ensure_ascii=False) + "\n")

def generate_response(instruction_text, input_text, max_tokens):
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction_text, # instruction
            input_text, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, 
                             max_new_tokens=max_tokens, 
                             use_cache=True, 
                             do_sample=True,
                             repetition_penalty=1.1,
                             no_repeat_ngram_size=4,
                             temperature=0.7,
                             top_p=0.9,
                             top_k=50, )
    decoded_outputs = tokenizer.batch_decode(outputs)

    response_texts = [output.split("### Response:\n")[-1].strip() for output in decoded_outputs]

    return response_texts[0].replace("<|eot_id|>", "")

# 실행 
input_json_path = "/data/jaesung/llm_for_diabetes/src/model/mLoRA/demo/data_generation_test.json"
output_json_path = "/data/jaesung/llm_for_diabetes/src/model/unsloth/model_output/generation_model_output.jsonl"

data = load_json(input_json_path)

start_time = time.time()
total_samples = len(data)
for idx, item in enumerate(data):
    sample_start_time = time.time()

    input_text = item.get("input", "")
    instruction = item.get("instruction", "")

    model_output_512 = generate_response(instruction, input_text, 512)
    # model_output_2048 = generate_response(instruction, input_text, 1024)

    output_data = item.copy()
    output_data.update({
        "model_output_512": model_output_512,
        # "model_output_2048": model_output_2048,
    })
    save_to_jsonl(output_json_path, output_data)

    elapsed_time = time.time() - start_time
    avg_time_per_sample = elapsed_time / (idx + 1) 
    remaining_samples = total_samples - (idx + 1)
    estimated_remaining_time = remaining_samples * avg_time_per_sample

    print(f"[{idx+1}/{total_samples}] Sample processed in {time.time() - sample_start_time:.2f}s, ETA: {estimated_remaining_time/60:.2f} min")

print(f"\nAll samples processed. Total time: {(time.time() - start_time)/60:.2f} min")

[1/720] Sample processed in 3.64s, ETA: 43.64 min
[2/720] Sample processed in 8.16s, ETA: 70.63 min
[3/720] Sample processed in 3.28s, ETA: 60.08 min
[4/720] Sample processed in 6.19s, ETA: 63.45 min
[5/720] Sample processed in 3.96s, ETA: 60.13 min
[6/720] Sample processed in 3.51s, ETA: 56.99 min
[7/720] Sample processed in 1.59s, ETA: 51.49 min
[8/720] Sample processed in 3.86s, ETA: 50.71 min
[9/720] Sample processed in 2.93s, ETA: 48.87 min
[10/720] Sample processed in 3.64s, ETA: 48.23 min
[11/720] Sample processed in 4.12s, ETA: 48.21 min
[12/720] Sample processed in 5.92s, ETA: 49.95 min
[13/720] Sample processed in 3.45s, ETA: 49.17 min
[14/720] Sample processed in 0.82s, ETA: 46.29 min
[15/720] Sample processed in 5.01s, ETA: 47.06 min
[16/720] Sample processed in 7.24s, ETA: 49.36 min
[17/720] Sample processed in 6.03s, ETA: 50.55 min
[18/720] Sample processed in 16.67s, ETA: 58.51 min
[19/720] Sample processed in 6.87s, ETA: 59.57 min
[20/720] Sample processed in 2.60s, ETA

In [20]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Given the previous meal, recommend the next meal that maintains a nutritionally balanced diet. The recommended meal must be selected only from the provided dataset and should not include any additional information.\n\nThe output must strictly follow this format:\n\n\"[Recommended meal] is recommended. The reason is: [Explanation]\"", # instruction
        "The previous food intake included: BaNonea, raw", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=512, use_cache=True)
decoded_outputs = tokenizer.batch_decode(outputs)

response_texts = [output.split("### Response:\n")[-1].strip() for output in decoded_outputs]

print(response_texts[0].replace("<|eot_id|>", "")) 


Black Bean and Corn Salad is recommended. The reason is 'Black Bean and Corn Salad' complements 'BaNonea, raw' as it provides a good source of protein, fiber, and potassium, which are lacking in the consumed food. However, caution should be taken with the sodium content in the salad, as it may be high for individuals with specific dietary needs.
